# Extract MODIS Site Data and Generate Samples
## MODIS Site Data
For each site in the LFMC sample data, extract the full time series of MODIS reflectance and snow-cover data, and save to CSV files. Note: if the output csv files already exist they are assumed to be correct and are not over-written. The data is gap-filled after being saved but before being used to create the MODIS sample data.
## MODIS Sample Data
For each sample, extract the timeseries MODIS reflectance data. The timeseries length is determined by the MODIS_TS_LENGTH value. The sample is rejected if the full timeseries cannot be extracted (start/end outside the full site time series). The extracted MODIS data is combined to a single dataframe and saved. Another LFMC sample dataset containing only the valid samples is also created.
## Input Files
- `LFMC_sites.csv` and `LFMC_samples.csv` created by the `Extract Auxiliary Data.ipynb` notebook.

## Output Files
- The extracted sites data are created in `MODIS_DIR` (by default `MCD43A4` located in `LFMC_DATA_DIR/GEE_DIR`). The directory will contain a CSV file for each site.
- The extracted MODIS data for each samples and updated samples data are created in `DATASETS_DIR`. File names include the time series length (i.e. 730days) of the extracted MODIS data. So, with the default settings they are `modis_730days.csv` and `samples_730days.csv`.

## Notes
1. This notebook should be run after running the `Extract Auxiliary Data.ipynb` notebook.
2. The generated name for `GEE_DIR` includes the projection and scale of the extracted GEE data. 
3. It will take about 8.5 hours to run if there are no existing site extracts.
4. 2364 samples will not be extracted as they were collected before March 2002 (so less than 2 years MODIS data available).
5. There should be no invalid sites, but occasionally extraction from GEE will fail for a site. If this happens re-run the notebook (keep the existing site CSV files so they are not re-extracted).

In [ ]:
import os
os.environ["HDF5_DISABLE_VERSION_CHECK"] = "1"
import numpy as np
import pandas as pd
import re
import time
from datetime import datetime
from datetime import timedelta

import initialise
import common
from data_extract_utils import get_sample_data, extract_timeseries_data, sort_key
from timeseries_extractor import GeeTimeseriesExtractor

### Program parameters and constants

GEE Parameters
- Reflectance product is MCD43A4 - daily reflectance using 8-day composites
- Scale set to use native MODIS resolution
- Extracts two years' of data for each sample to allow for a 1-year time series with a 1-year lead time

In [ ]:
# MODIS time series constants
MODIS_TS_LENGTH = 730
MODIS_TS_OFFSET = 1
MODIS_TS_FREQ = 1

# MODIS data details
PRODUCT = "MODIS/006/MCD43A4"
BANDS = ["Nadir_Reflectance_Band1",
         "Nadir_Reflectance_Band2",
         "Nadir_Reflectance_Band3",
         "Nadir_Reflectance_Band4",
         "Nadir_Reflectance_Band5",
         "Nadir_Reflectance_Band6",
         "Nadir_Reflectance_Band7"]

EARLIEST_SAMPLE = datetime.strptime(common.START_DATE, '%Y-%m-%d') + timedelta(
    days=MODIS_TS_LENGTH * MODIS_TS_FREQ + MODIS_TS_OFFSET)

### Directories and Files

In [ ]:
# Sub-directories for GEE extracts
GEE_DIR = f"GEE_{common.PROJ.replace(':', '-')}_{int(common.SCALE)}"
MODIS_DIR = os.path.join(common.LFMC_DATA_DIR, GEE_DIR, "MCD43A4")

# File Names
SAMPLES_OUTPUT = os.path.join(common.DATASETS_DIR, f"samples_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")
MODIS_OUTPUT = os.path.join(common.DATASETS_DIR, f"modis_{MODIS_TS_LENGTH * MODIS_TS_FREQ}days.csv")

# Create output directories if necessary
if not os.path.exists(MODIS_DIR):
    os.makedirs(MODIS_DIR)

## Main Processing
Connect to GEE

In [ ]:
import ee
ee.Initialize()

### Generate MODIS sample data

For each site, get the sample data for each sample at the site

Note: This gets the data using the default MODIS scale and projection. To change the scale and/or projection, add calls to the `GeeTimeseriesExtractor.setProjScale` method

In [ ]:
sites = pd.read_csv(common.LFMC_SITES, float_precision="high")
samples = pd.read_csv(common.LFMC_SAMPLES, float_precision="high")
modis_extractor = GeeTimeseriesExtractor(PRODUCT, BANDS, common.START_DATE, common.END_DATE,
                                         dir_name=MODIS_DIR)
modis_extractor.set_proj_scale(common.PROJ, common.SCALE)
modis_data, valid_data, invalid_pixels, invalid_sites = extract_timeseries_data(
    modis_extractor, sites, samples, EARLIEST_SAMPLE, MODIS_TS_OFFSET, MODIS_TS_LENGTH, MODIS_TS_FREQ)
# modis_data = []
# valid_data = [False] * samples.shape[0]
# invalid_pixels = []
# invalid_sites = []
# for site_idx, site in sites.iterrows():
#     print(f'Processing site {site.Site}')
#     site_samples = samples[samples.Site == site.Site]
#     try:
#         modis_df = modis_extractor.get_and_save_data(site)
#     except:
#         print(f'Failed to extract data for {site.Site}')
#         invalid_sites.append(site.Site)
#         continue
#     for index, sample in site_samples.iterrows():
#         sample_data = get_sample_data(sample["Sampling date"], modis_df,
#                                       MODIS_TS_OFFSET, MODIS_TS_LENGTH, MODIS_TS_FREQ)
#         if sample_data is None or np.isnan(sample_data.sum()):
#             invalid_pixels.append(index)
#         else:
#             modis_data.append([sample.ID] + list(sample_data))
#             valid_data[index] = True

Summary of sites/samples not extracted

In [ ]:
print(f'Invalid sites: {len(invalid_sites)}; Invalid pixels: {len(invalid_pixels)}; ')
print(invalid_sites)
print(invalid_pixels)

### Save Results
Save and display sample reflectance data

In [ ]:
modis_data = pd.DataFrame(modis_data)
ts_days = (MODIS_TS_LENGTH - 1) * MODIS_TS_FREQ
modis_data.columns = ["ID"] + [f'{day-MODIS_TS_OFFSET:04}_{band+1}'
                               for day in range(-ts_days, 1, MODIS_TS_FREQ)
                               for band in range(len(BANDS))]
modis_data.sort_values('ID', inplace=True, key=lambda x: x.apply(sort_key))
modis_data.to_csv(MODIS_OUTPUT, index=False)
modis_data

Save and display the valid samples

In [ ]:
valid_samples = samples[valid_data].sort_values('ID', key=lambda x: x.apply(sort_key))
valid_samples.to_csv(SAMPLES_OUTPUT, index=False)
valid_samples